# Тестирование модуля формирования портфеля

Этот ноутбук демонстрирует полный цикл работы с модулем 	vr_service.pipeline.portfolio — от загрузки данных по инструментам до расчёта распределения капитала и экспорта результатов.

## 1. Подготовка окружения

Импортируем необходимые библиотеки и подключим модуль портфеля. Добавляем каталог src в sys.path, чтобы можно было использовать код проекта без установки пакета.

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
SRC_DIR = PROJECT_ROOT / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

from tvr_service.pipeline import (
    allocations_to_frame,
    build_portfolio,
    filter_by_suffix,
    load_securities,
    load_whitelist,
)

import pandas as pd


## 2. Настройка параметров теста

Задаём базовые параметры: объём капитала, необходимость скачивания свежих данных с MOEX, путь к локальному CSV (если он есть) и фильтрацию по суффиксу тикера. Все параметры можно менять и перезапускать ячейки.

In [ ]:
CAPITAL = 1_000_000  # общий капитал, руб.
PREFER_REMOTE = True   # при True сначала пробуем скачать данные с MOEX
SEC_DATA_FILE = None   # можно указать путь к локальному sec_tvr.csv
SUFFIX_FILTER = None   # например, 'H5', 'M5' и т.п. для фьючерсов


## 3. Загрузка справочника инструментов

Попробуем получить таблицу инструментов. Функция load_securities автоматически рассчитает поле ull_price и добавит тикеры TI* (клонов TB*). В процессе будет использован список из интернета или локального файла.

In [ ]:
securities = load_securities(sec_data_file=SEC_DATA_FILE, prefer_remote=PREFER_REMOTE)
securities.head()

### Новые поля справочника

Справочник теперь содержит дополнительные столбцы:

* `SHORTNAME` — полное имя инструмента, как его отдаёт ISS.
* `CODE` — копия `SECID` (для совместимости со старым кодом).
* `PRICE` и `SELLDEPO` — котировка и гарантийное обеспечение для расчётов.
* `base_code` — базовое имя (до дефиса), используется при фильтрации whitelist.

In [ ]:
securities[['SECID', 'SHORTNAME', 'base_code', 'PRICE', 'SELLDEPO']].head()

## 4. Проверка whitelist

Whitelist может хранить как полные имена, так и базовые коды. Ниже показан пример фильтрации по новым колонкам.

In [ ]:
try:
    whitelist = load_whitelist()
    print(f'Инструментов в whitelist: {len(whitelist)}')
except FileNotFoundError:
    whitelist = None
    print('Файл whitelist не найден. Будем использовать полную вселенную инструментов.')

if whitelist:
    filtered_by_whitelist = filter_by_whitelist(securities, whitelist)
    print(f'Инструментов после применения whitelist: {len(filtered_by_whitelist)}')
    print('Примеры совпадений:')
    print(filtered_by_whitelist[['SECID', 'SHORTNAME', 'base_code']].head())
else:
    filtered_by_whitelist = securities


## 5. Построение портфеля

Выполним расчёт: функция uild_portfolio вернёт список PortfolioEntry, содержащий распределённый капитал и оценку количества лотов. Затем конвертируем результат в DataFrame для удобного анализа.

In [ ]:
portfolio_entries = build_portfolio(
    capital=CAPITAL,
    suffix=SUFFIX_FILTER,
    whitelist=whitelist,
    sec_data_file=SEC_DATA_FILE,
    prefer_remote=PREFER_REMOTE,
)

portfolio_df = allocations_to_frame(portfolio_entries)
portfolio_df.head()

## 6. Диагностика распределения капитала

Проверим, сколько средств было фактически распределено по инструментам, и какой объём остался неиспользованным из-за округления до целых лотов.

In [ ]:
allocated_capital = portfolio_df['used_capital'].sum()
unused_capital = portfolio_df['unused_capital'].sum()
print(f'Фактически распределено: {allocated_capital:,.2f} руб.')
print(f'Неиспользованный остаток: {unused_capital:,.2f} руб.')
print(f'Доля остатка: {unused_capital / CAPITAL:.2%}')


## 7. Топ инструментов по задействованному капиталу

Отсортируем таблицу по полю used_capital, чтобы понять, какие инструменты потребляют наибольшую долю капитала.

In [ ]:
portfolio_df.sort_values('used_capital', ascending=False).head(10)

## 8. Сохранение результатов

По желанию можно сохранить расчёт в CSV. По умолчанию файлы складываются в каталог docs.

In [ ]:
output_path = PROJECT_ROOT / 'docs' / 'portfolio_result.csv'
portfolio_df.to_csv(output_path, index=False)
output_path

## 9. Следующие шаги

* Изменяйте параметры и фильтры, чтобы тестировать разные сценарии.
* Добавьте собственные таблицы параметров, когда они будут готовы, и объедините их с расчётами.
* Используйте ноутбук как основу для презентации результатов или автоматизации отчётов.